### Домашнее задание Transformers Training (50 баллов)

В этом домашнем задании требуется обучить несколько Transformer-based моделей в задаче машинного перевода. Для обучения можно воспользоваться текущим проектом, так и реализовать свой пайплайн обучения. Если будете использовать проект, теги **TODO** проекта отмечают, какие компоненты надо реализовать.
В ноутбуке нужно только отобразить результаты обучения и выводы. Архитектура модели(количетсво слоев, размерность и тд) остается на ваш выбор.

Ваш код обучения нужно выложить на ваш github, в строке ниже дать ссылку на него. В первую очередь будут оцениваться результаты в ноутбуке, код нужен для проверки адекватности результатов. 

Обучать модели до конца не нужно, только для демонстрации, что модель обучается и рабочая - снижение val_loss, рост bleu_score.

#### Сcылка на github с проектом - 

Ноутбук с результатами выкладывать на ваш **google диск** курса. 

### Данные

`
wget https://www.manythings.org/anki/rus-eng.zip && unzip rus-eng.zip
`

Модели нужно обучить на задаче перевода с английского на русский. 

### Обучение Seq2seq Transformer модель(25 баллов)

Реализуйте Seq2seq Transformer. В качестве блока трансформера можно использовать https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html. В качестве токенизатора воспользуйтесь HuggingFace токенизатор для source/target языков - https://huggingface.co/docs/transformers/fast_tokenizers
В качестве максимальной длинны возьмите предложения длинной **до 15 слов**, без каких либо префиксов. 

Не забудьте остальные элементы модели:
* Мы можем использовать 1 трансформер как энкодер - декодером будет выступать линейный слой. 
* Обучите свой BPE токенизатор - https://huggingface.co/docs/transformers/fast_tokenizers
* Матрицу эмбеддингов токенов
* Матрицу позицонных эмбеддингов
* Линейный слой проекции в target словарь
* Функцию маскирования будущих состояний attention, так как модель авто-регрессионна
* Learning rate schedualer


В качестве результатов, приложите слудующие данные:
1) Параметры обучения - learning rate, batch_size, epoch_num, размерность скрытого слоя, количетсво слоев
2) Графики обучения - train loss, val loss, bleu score
3) Примеры переводов вашей модели(10 штук) - source text, true target text, predicted target text

In [ ]:
# !wget https://www.manythings.org/anki/rus-eng.zip && unzip rus-eng.zip
# !pip install tokenizers

In [1]:
import sys
sys.path.insert(0, '/ssd/r.musaev/DL_sber_hw/pytorch-machine-translation/src')

In [2]:
import torch
import numpy as np
import random
import torch.nn as nn
import yaml
from src.data.datamodule import DataManager
from src.txt_logger import TXTLogger
%reload_ext autoreload
%autoreload 2
from src.models.seq2seq_transformer import Seq2SeqTransformer
from src.models import trainer

ModuleNotFoundError: No module named 'torchtext'

## fix seed and define boilerplate

In [84]:
def seed_everything(seed: int):    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
logger = TXTLogger("baseline")

## prepare data

In [85]:
# with open("configs/data_config.yaml", "r") as fread:
#     data_config = yaml.load(fread, Loader=yaml.Loader)

# data_manager = DataManager(data_config, device)
# train_dataloader, val_dataloader = data_manager.prepare_data()
# target_tokenizer = data_manager.target_tokenizer


## prepare model

In [86]:
with open("configs/model_config.yaml", "r") as fread:
    model_config = yaml.load(fread, Loader=yaml.Loader)

model_config["src_vocab_size"] = data_manager.source_tokenizer.tokenizer.get_vocab_size()
model_config["tgt_vocab_size"] = data_manager.target_tokenizer.tokenizer.get_vocab_size()
model_config["T_max"] = model_config["epoch_num"] * len(train_dataloader)
model_config["device"] = device 

print(model_config["tgt_vocab_size"], model_config["src_vocab_size"])

model = Seq2SeqTransformer(model_config, target_tokenizer.word2index).to(device)

4313 2703


In [87]:
trainer_cls = trainer.Trainer(
    model=model,
    model_config=model_config,
    logger=logger,
    target_tokenizer=target_tokenizer
)

trainer_cls.train(train_dataloader, val_dataloader)

  0%|          | 0/5000 [00:00<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "/netapp/r.musaev/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2172084/3436850606.py", line 8, in <cell line: 8>
    trainer_cls.train(train_dataloader, val_dataloader)
  File "/ssd/r.musaev/DL_sber_hw/pytorch-machine-translation/src/models/trainer.py", line 20, in train
    train_loss = self.model.training_step(batch)
  File "/ssd/r.musaev/DL_sber_hw/pytorch-machine-translation/src/models/seq2seq_transformer.py", line 90, in training_step
    decoder_outputs = self.forward(input_tensor, target_tensor, src_mask=src_mask, tgt_mask=tgt_mask, src_key_padding_mask=src_key_padding_mask, tgt_key_padding_mask=tgt_key_padding_mask)
  File "/ssd/r.musaev/DL_sber_hw/pytorch-machine-translation/src/models/seq2seq_transformer.py", line 73, in forward
    out = self.transformer_model(src_embed,
  File "/ssd/r.musaev/python/miniconda3

In [258]:
with open("./baseline/progress_log.txt", 'r') as fread:
    stats = fread.readlines()

In [259]:
stats

["{'embedding_size': 100, 'hidden_size': 100, 'learning_rate': 0.001, 'epoch_num': 5000, 'try_one_batch': True, 'nhead': 1, 'num_encoder_layers': 4, 'lr': 0.01, 'weight_decay': 0.0005, 'maxlen': 15, 'src_vocab_size': 2703, 'tgt_vocab_size': 4313, 'T_max': 85000, 'device': 'cuda'}\n"]

In [ ]:
for i in range(10):
    src_decoded = " ".join(data_manager.source_tokenizer.decode(inp[i].cpu().numpy().tolist()))
    out_decoded = " ".join(data_manager.target_tokenizer.decode(out[i].cpu().numpy().tolist()))
    tgt_decoded = " ".join(data_manager.target_tokenizer.decode(tgt[i].cpu().numpy().tolist()))
    print(f"{1 + i}.\tEN: {src_decoded}\n\tRU: {tgt_decoded}\n\tPRED: {out_decoded}")

### Fine-tune pretrained T5 (25 баллов)

Реализуйте Seq2seq Pretrained T5. Воспользуйтесь https://huggingface.co/docs/transformers/model_doc/t5 предобученной моделью. В качестве максимальной длинны возьмите предложения длинной **до 15 слов**, без каких либо префиксов. Архитектура модели(количетсво слоев, размерность и тд) остается на ваш выбор.

Не забудьте важные аспекты обучения модели:
* Взять готовый t5 токенизатор
* Resize matrix embedding - скорей всего ваша матрица эмбеддингов не будет включать эмбеддинги из вашего сета. Пример обновления матрицы эмбеддингов тут тут https://github.com/runnerup96/Transformers-Tuning/blob/main/t5_encoder_decoder.py
* Learning rate schedualer/Adafactor with constant learning rate


В качестве результатов, приложите слудующие данные:
1) Параметры обучения - learning rate, batch_size, epoch_num, pretrained model name
2) Графики обучения - train loss, val loss, bleu score
3) Примеры переводов вашей модели(10 штук) - source text, true target text, predicted target text

In [ ]:
# CODE HERE